# 生成されたサンプルを辞書に基づいて日本語に直す

seqGANで生成されたサンプルを辞書に基づいて日本語の文字列に変換する

また、変換する際には、seqGANの識別機が算出したサンプルに対する本物かどうかの確率と紐付ける

これは、生成されたキャッチコピーに対する自然さの度合いを表していると考えられ、キャッチコピーに対するスコア付けの手法の一つとして用いる

In [ ]:
# 生成されたサンプルを辞書に基づいて日本語に戻す
import pickle

with open('generated/sepa/dazaifu/content/save/output_text_300.txt') as f:
    s = f.readlines()

with open('generated/sepa/dazaifu/content/save/prob_300.txt') as f:
    prob = f.readlines()

with open('data/dict/kankouti_dict.pickle', 'rb') as f:
    indices_char = pickle.load(f)

temp_list = []

for i, j in zip(s, prob):
    temp = [i,j]
    temp_list.append(temp)

print("生成数",len(temp_list))

num_prob_list = []

for k in temp_list:
    if k not in num_prob_list:
        num_prob_list.append(k)

print("重複をなくした生成数",len(num_prob_list))




# # 順番を保持したまま重複をなくす
# s = sorted(set(s), key=s.index)
# print(len(s))

haiku_prob_list = []
haiku_temp = []
result = ""
for j in range(len(num_prob_list)):
    line = num_prob_list[j][0].split()
    line = [int(i) for i in line]
    # print(line)
    # 単語に対応するidを並べて行く
    for i in line:
        if i > len(indices_char):
            continue
        # print(char_indices[i])
        result += str(indices_char[i])
        result += " "
    # result += "0" # 文末記号を追加
    result = result.replace('_', '')
    result = result.replace(' ', '')
    # result += "\n"
    haiku_temp.append(result)
    result = ""
# print(haiku_temp[:10])
for p in range(len(haiku_temp)):
    num_prob_list[p][1] = num_prob_list[p][1][:-1]
    temp = [haiku_temp[p],num_prob_list[p][1]]
    haiku_prob_list.append(temp)

print(len(haiku_prob_list))
print("\n")
print(haiku_prob_list[:10])


日本語に変換したサンプルが575の音韻を満たしているかフィルタリングを行う

やってること自体は、レビューから575の文字列を抽出しているのと同じ

In [ ]:
# with open('generated/sepa/hakataeki/content/save/output/output_2_haiku.txt') as f:
#     s = f.readlines()

import MeCab
tagger = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd") #mecab neologd


# text_list = []

# for line in s:
#     line = line.replace(" ", "")
#     text_list.append(line)

# print(text_list)




haiku_list = []
for x,y in haiku_prob_list:
    temp = x
    # print(temp)

    word_list = []

    # 形態素解析をして、単語毎にリストに格納
    for word in tagger.parse(temp).splitlines():
        word = word.replace('\t', ',')
        # print(word)
        temp_word_list = word.split(',')
        word_list.append(temp_word_list)

    # 文末の記号を削除
    word_list.pop(-1)

    # print(word_list)

    # 格納された単語を繋げて17音の文字列を作る
    for j in range(len(word_list)):

        # 最初に空の文字列を用意
        temp_haiku = ""
        part = ""
        # 次の単語を示すカウント
        count = 0
        # 音を数えるcount
        haku_count = 0

        # 17音を超えるまで単語を繋げる

        if not ('名詞' == word_list[j+count][1] or '動詞' == word_list[j+count][1] or '形容詞' == word_list[j+count][1]):
            continue
        if ('名詞' ==  word_list[j+count][1] and ('非自立' == word_list[j+count][2] or '接尾' == word_list[j+count][2])):
            continue
        if ('動詞' ==  word_list[j+count][1] and '非自立' == word_list[j+count][2]):
            continue

        # 上の句5
        while haku_count < 5:
            # 単語を繋げる
            part =part + word_list[j + count][0]
            # 音を記録

            if '記号' not in word_list[j+count][1]:
                temp = word_list[j+count][-1]

                # 小さい文字は音にならないので消去
                # temp = temp.replace('ッ', '')
                temp = temp.replace('ャ', '')
                temp = temp.replace('ュ', '')
                temp = temp.replace('ョ', '')
                temp = temp.replace('ァ', '')
                temp = temp.replace('ィ', '')
                temp = temp.replace('ゥ', '')
                temp = temp.replace('ェ', '')
                temp = temp.replace('ォ', '')

                haku_count += len(temp)
            # 元の文が終わっていたら終了
            if j + count >= len(word_list)-1:
                break
            count += 1
            # print(part, haku_count)
        
        if haku_count == 5:
            temp_haiku += part
            # temp_haiku += "\n"
            part = ""
            haku_count = 0
        else:
            continue

        if not ('名詞' == word_list[j+count][1] or '動詞' == word_list[j+count][1] or '形容詞' == word_list[j+count][1]):
            continue
        if ('名詞' ==  word_list[j+count][1] and ('非自立' == word_list[j+count][2] or '接尾' == word_list[j+count][2])):
            continue
        if ('動詞' ==  word_list[j+count][1] and '非自立' == word_list[j+count][2]):
            continue

        # 中の句7
        while haku_count < 7:
            # 単語を繋げる
            part =part + word_list[j + count][0]
            # 音を記録

            if '記号' not in word_list[j+count][1]:
                temp = word_list[j+count][-1]

                # 小さい文字は音にならないので消去
                # temp = temp.replace('ッ', '')
                temp = temp.replace('ャ', '')
                temp = temp.replace('ュ', '')
                temp = temp.replace('ョ', '')
                temp = temp.replace('ァ', '')
                temp = temp.replace('ィ', '')
                temp = temp.replace('ゥ', '')
                temp = temp.replace('ェ', '')
                temp = temp.replace('ォ', '')

                haku_count += len(temp)
            # 元の文が終わっていたら終了
            if j + count >= len(word_list)-1:
                break
            count += 1
            # print(temp_haiku, haku_count)

        
        
        if haku_count == 7:
            temp_haiku += part
            # temp_haiku += "\n"
            part = ""
            haku_count = 0
        else:
            continue

        if not ('名詞' == word_list[j+count][1] or '動詞' == word_list[j+count][1] or '形容詞' == word_list[j+count][1]):
            continue
        if ('名詞' ==  word_list[j+count][1] and ('非自立' == word_list[j+count][2] or '接尾' == word_list[j+count][2])):
            continue
        if ('動詞' ==  word_list[j+count][1] and '非自立' == word_list[j+count][2]):
            continue

        # 下の句5
        while haku_count < 5:
            # 単語を繋げる
            part =part + word_list[j + count][0]
            # 音を記録

            if '記号' not in word_list[j+count][1]:
                temp = word_list[j+count][-1]

                # 小さい文字は音にならないので消去
                # temp = temp.replace('ッ', '')
                temp = temp.replace('ャ', '')
                temp = temp.replace('ュ', '')
                temp = temp.replace('ョ', '')
                temp = temp.replace('ァ', '')
                temp = temp.replace('ィ', '')
                temp = temp.replace('ゥ', '')
                temp = temp.replace('ェ', '')
                temp = temp.replace('ォ', '')

                haku_count += len(temp)
            # 元の文が終わっていたら終了
            if j + count >= len(word_list)-1:
                break
            count += 1
            # print(temp_haiku, haku_count)

        
        if haku_count == 5:
            temp_haiku += part
            # temp_haiku += "\n"
            # temp_haiku += "\n"
            part = ""
            haku_count = 0
        else:
            continue
        

        if temp_haiku not in haiku_list:
            haiku_list.append([temp_haiku,y])

print(len(haiku_list))

print(haiku_list[:10])


# スコア付け

# tf-idfを用いたスコア付け

フィルタリングを行ったサンプルに対して、３つの手法でスコア付けを行う

tf-idfを用いたスコア付けでは、サンプルに含まれる名詞に対するtf-idfの値の合計をキャッチコピーのスコアとする

これはキャッチコピーが持つ観光地の特徴度合いを表していると考えられる

In [ ]:
#-------------------------------------------------データベース----------------------------------------

import sqlite3

db_word_list = []

conn = sqlite3.connect('kankouti_tfidf_db.db')

c = conn.cursor()

c.execute('select name from dazaifu')
db_words = c.fetchall()

for word in db_words:
    db_word_list.append(word[0])

# print(db_word_list)

#-------------------------------------------------tfidfのスコア付け----------------------------------------
score_list = []

for i,_ in haiku_list:
    temp = i
    # print(temp)

    word_list = []
    score_sum = 0

    # 形態素解析をして、単語毎にリストに格納
    for word in tagger.parse(temp).splitlines():
        word = word.replace('\t', ',')
        temp_word_list = word.split(',')
        word_list.append(temp_word_list)

    # 文末の記号を削除
    word_list.pop(-1)

    

    for j in range(len(word_list)):
        # print(word_list[j][0])

        temp_score = 0

        if '名詞' not in word_list[j][1]:
            continue

        # 形態素解析した名詞がdbに含まれているならtfidfの値をスコアとして加算していく
        if word_list[j][0] in db_word_list:
            c.execute('select tfidf from dazaifu where name = ?', (word_list[j][0],))
            tfidf = c.fetchone()[0]
            temp_score = tfidf
        
        score_sum += temp_score

    score_list.append(score_sum)

        

print(score_list)

#-------------------------------------------------データベース----------------------------------------

#閉じる
conn.close()



